In [1]:
import google.generativeai as genai

import pandas as pd
import pickle
import os

import re

In [2]:
base_save_folder_dir = '../saved/'
dataset_folder = os.path.join(base_save_folder_dir, 'Dataset')

with open(os.path.join(dataset_folder, 'dataset_cleaned.pkl'), 'rb') as f:
    dataset = pickle.load(f)

In [3]:
dataset

,language,article_id,content,narratives,subnarratives,narratives_encoded,subnarratives_encoded
0,RU,RU-URW-1161.txt,<PARA>в ближайшие два месяца сша будут стремит...,[Blaming the war on others rather than the inv...,"[The West are the aggressors, Other, The West ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,RU,RU-URW-1175.txt,<PARA>в ес испугались последствий популярности...,"[Discrediting the West, Diplomacy, Discreditin...","[The West is weak, Other, The EU is divided]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,RU,RU-URW-1149.txt,<PARA>возможность признания аллы пугачевой ино...,[Distrust towards Media],[Western media is an instrument of propaganda],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,RU,RU-URW-1015.txt,<PARA>азаров рассказал о смене риторики киева ...,"[Discrediting Ukraine, Discrediting Ukraine]","[Ukraine is a puppet of the West, Discrediting...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,RU,RU-URW-1001.txt,<PARA>в россиянах проснулась массовая любовь к...,[Praise of Russia],[Russia is a guarantor of peace and prosperity],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1694,EN,EN_UA_008072.txt,"<PARA>Putin Mobilizes more troops, hints at nu...",[Amplifying war-related fears],[There is a real possibility that nuclear weap...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1695,EN,EN_CC_300151.txt,<PARA>climate expert: ‘three strikes’ against ...,"[Criticism of climate movement, Downplaying cl...","[Climate movement is alarmist, Other, Climate ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1696,EN,EN_CC_200145.txt,<PARA>GOP slams John Kerry's Hypocrisy on coal...,"[Criticism of institutions and authorities, Co...",[Criticism of political organizations and figu...,"[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1697,EN,EN_UA_015962.txt,<PARA>dramatic moment anti-Putin troops ‘invad...,[Speculating war outcomes],[Russian army is collapsing],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
def concatenate_paragraphs(content):
    parts = re.split(r'<PARA>|</PARA>', content)
    parts = [p.strip() for p in parts if p.strip()]
    return " ".join(parts)

In [13]:
def truncate_to_bytes(input_string, max_bytes=10000, encoding='utf-8'):

    encoded_bytes = input_string.encode(encoding)
    truncated_bytes = encoded_bytes[:max_bytes]
    truncated_string = truncated_bytes.decode(encoding, errors='ignore')
    
    return truncated_string

In [14]:
from tqdm.auto import tqdm
tqdm.pandas()

max_bytes = 9960

from google.api_core import retry

def make_embed_text_fn(model):
    
    @retry.Retry(timeout=300.0)
    def embed_fn(text: str) -> list[float]:
        embedding = genai.embed_content(
            model=model,
            content=truncate_to_bytes(
                concatenate_paragraphs(text), 
                max_bytes=max_bytes),
            task_type="classification")
        return embedding['embedding']

    return embed_fn

In [15]:
def create_embeddings(model, df):
    df['embedding'] = df['content'].progress_apply(make_embed_text_fn(model))
    return df

In [16]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [17]:
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


In [18]:
model = 'models/text-embedding-004'

dataset = create_embeddings(model, dataset)

  0%|          | 0/1699 [00:00<?, ?it/s]

InvalidArgument: 400 Request payload size exceeds the limit: 10000 bytes.

In [22]:
dataset[dataset['content'].str.len() > 10000]

,language,article_id,content,narratives,subnarratives,narratives_encoded,subnarratives_encoded
957,HI,HI_83.txt,<PARA>pm मोदी के यूक्रेन दौरे में छिपे हैं 'यु...,[Discrediting Ukraine],[Ukraine is a puppet of the West],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
976,HI,HI_117.txt,<PARA>संयुक्त रिपोर्ट: यूक्रेन युद्ध का विभिन्...,"[Overpraising the West, Negative Consequences ...",[The West has the strongest international supp...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1122,HI,HI_208.txt,<PARA>हिलते पहाड़ः जलवायु परिवर्तन का अपक्षय औ...,"[Downplaying climate change, Criticism of clim...",[Temperature increase does not have significan...,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, ..."
1254,HI,HI_66.txt,<PARA>संयुक्त रिपोर्ट: यूक्रेन युद्ध का विभिन्...,"[Negative Consequences for the West, Negative ...",[Sanctions imposed by Western countries will b...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [28]:
len(truncate_to_bytes(dataset.iloc[957]['content']).encode('utf-8'))

9997

In [39]:
text = dataset.iloc[957]['content']
genai.embed_content(
            model=model,
            content=truncate_to_bytes(
                concatenate_paragraphs(text), max_bytes=9960),
            task_type="classification")

{'embedding': [-0.000116944124,
  0.01583753,
  -0.0633087,
  0.017454736,
  0.081481054,
  0.042934712,
  -0.02412606,
  0.013869079,
  0.037926126,
  -0.012866721,
  0.03131457,
  0.08401634,
  0.09754606,
  0.04469645,
  0.08816097,
  -0.01940392,
  0.04408309,
  0.02668023,
  -0.045191884,
  -0.032719616,
  -0.022813633,
  0.006509582,
  0.025683207,
  0.030369248,
  -0.012969774,
  -0.049620353,
  0.0012999633,
  -0.0042469474,
  0.027345162,
  -0.04600582,
  0.0034636918,
  0.052471474,
  0.0077236486,
  0.009708997,
  -0.00050768076,
  0.047972914,
  0.045251008,
  0.014664726,
  -0.02558562,
  -0.08445494,
  -0.047572337,
  -0.02977365,
  -0.007032853,
  0.10602843,
  -0.01780087,
  -0.039741028,
  0.033410817,
  -0.03228833,
  0.0013223642,
  0.026119923,
  -0.0018388783,
  -0.056564573,
  0.025897803,
  0.020468345,
  -0.051106185,
  -0.031803187,
  -0.0493032,
  -0.04513077,
  0.042780135,
  -0.026787192,
  -0.046795588,
  0.009635079,
  0.015624526,
  -0.06489681,
  0.01366